In [36]:
# | default_exp agent_coder

## Setup

In [37]:
# | export

from langgraph.graph import StateGraph, END, START, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.base import BaseStore
from langchain_core.messages import AIMessage
from langchain_core.runnables import RunnableConfig
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.store.memory import InMemoryStore
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
)
from langchain_core.output_parsers.string import StrOutputParser
from pydantic import BaseModel, Field
from typing import List, Annotated, Literal, TypedDict, Optional
from operator import add, attrgetter
import textwrap
import os
from trustcall import create_extractor
import re
from pathlib import Path
import uuid

## Prompts

In [38]:
# | export

problem_distiller_prompt = textwrap.dedent(
    """
    ## Problem Distiller

    You are a highly professional and intelligent expert in information distillation. Your role is to extract essential information from user input queries to solve problems effectively. You also transform this extracted information into a suitable format based on the type of issue.

    ---

    ### Task Instructions

    1. **Key Information**:
    - Extract values and key variables from the user input.
    - Ensure all essential information required to solve the problem is provided.
    - Hand over this distilled information to the respective expert for task resolution.

    2. **Restrictions**:
    - Identify the objective of the problem.
    - Outline any corresponding constraints that must be adhered to.

    3. **Distilled Task**:
    - Extend the problem based on the extracted key information and constraints.
    - Summarize a meta problem that addresses the user query and accommodates more input and output variations.
    - Incorporate the real-world scenario of the extended problem.
    - Define types of key variables and information constraints from the original problem to restrict variables in the extended problem.
    - Use the input key information from the user query as an example to solve the problem.
    """
).strip()


instantiate_reasoning_prompt = textwrap.dedent(
    """
    ## Meta Reasoner

    You are a Meta Reasoner who is extremely knowledgeable in various fields, including Computer Science, Math, Physics, Literature, History, Chemistry, Logical Reasoning, Culture, and Language. You are also skilled in applying high-level reasoning structures for different tasks. 

    ### Reasoning Structures:

    1. **Prompt-based Structure**:
    - **Best For**: Common Sense Reasoning, Application Scheduling.
    
    2. **Procedure-based Structure**:
    - **Best For**: Creative tasks like Creative Language Generation, and Text Comprehension.
    
    3. **Programming-based Structure**:
    - **Best For**: Mathematical Reasoning, Code Programming.
    - Can transform real-world problems into programming problems to solve efficiently.

    ---

    ### Reasoning Instantiation

    **Your Task:**

    1. **Contextual Analysis**: Deliberately consider the context and the problem distilled from the problem distiller. Use your understanding to identify a suitable domain expert for solving the problem.

    2. **Structure Selection**: Based on the distilled information, select one of the reasoning structures suitable for addressing the problem.

    3. **Template Application**: If a thought-template is provided, directly follow it to instantiate the solution for the given problem.
    """
).strip()


template_distiller_prompt = textwrap.dedent(
    """
    ## Prompt for Template Distillation:

    **User Input**:
    **Problem Description**: {distilled_problem}
    **Solution Steps or Code**: {solution_steps}

    1. **Core task summarization**:
    Identify and describe the basic type and core challenges of the problem, such as classifying it as a mathematical problem (e.g., solving a quadratic equation), a data structure problem (e.g., array sorting), an algorithm problem (e.g., search algorithms), etc. And analyze the most efficient way to solve the problem.

    2. **Solution Steps Description**:
    Outline the general solution steps, including how to define the problem, determine variables, list key equations or constraints, choose appropriate solving strategies and methods, and how to verify the correctness of the results.

    3. **General Answer Template**:
    Based on the above analysis, propose a template or approach that can be widely applied to this type of problem, including possible variables, functions, class definitions, etc. If it is a programming problem, provide a set of base classes and interfaces that can be used to construct solutions to specific problems.

    Please ensure that your response is highly concise and structured, so that specific solutions can be transformed into generalizable methods.

    [Optional] Here are some exemplars of the thought-template:

    <in-task-examples>
    {in_task_examples}
    <in-task-examples>

    <cross-task-examples>
    {cross_task_examples}
    <cross-task-examples>
    """
)

structure_prompt = textwrap.dedent(
    """
    Extract the items of the 'ThoughtTemplate' Pydantic class from the previous conversation.
                                    
    <convo>
    {conversation}
    </convo> 
    """
)

## Templates

In [39]:
# | export


class ThoughtTemplate(BaseModel):
    """Defining the three fields of the Thought Template"""
    description: str = Field(description="Task Description")
    solution: str = Field(description="Solution Description")
    template: str = Field(description="Thought Template")

## Runnables

In [40]:
# | export

# Setup Large Language Model (LLM)
LLM = ChatOpenAI(
    model_name="gpt-4o-mini",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.0,
)

In [ ]:
# | export

# Define Trustcall instance for sane extraction of the Thought Template items: Task Description, Solution Description, Thought Template
structure_template_text = create_extractor(
    LLM, tools=[ThoughtTemplate], tool_choice="ThoughtTemplate")

## Long-Term Memory

In [41]:
# | export

# Create store with semantic search enabled
template_store = InMemoryStore(
    index={
        "embed": OpenAIEmbeddings(model="text-embedding-3-small"),
        "dims": 1536,
        "fields": ['task_description']
    }
)

Two helper functions to process the memory items by translating a markdown into a JSON following the ThoughtTemplate pattern and vice versa.

In [42]:
# | export

def markdown_to_json(markdown_text: str) -> dict:
    """
    Splits a markdown file along its ## headers and organizes content in a dictionary.

    Args:
        markdown_text: The markdown file content as a string.

    Returns:
        A dictionary where keys are transformed headers and values are paragraph text.
    """

    # Regular expression to match `##` headers
    header_pattern = re.compile(r"^##\s*(.+)$", re.MULTILINE)

    # Find all headers and their positions
    headers = [
        (match.start(), match.group(1))
        for match in header_pattern.finditer(markdown_text)
    ]

    # Dictionary to store the processed headers and content
    content_dict = {}

    for i, (start_pos, header) in enumerate(headers):
        # Determine the start and end of the content for this header
        end_pos = headers[i + 1][0] if i + \
            1 < len(headers) else len(markdown_text)

        # Extract content for the current header
        content = markdown_text[start_pos:end_pos]

        # Remove the header itself from the content
        content = header_pattern.sub("", content, count=1).strip()

        # Transform header: casefold, strip whitespace, and replace spaces with underscores
        transformed_header = header.casefold().strip().replace(" ", "_")

        # Add the header and its corresponding content to the dictionary
        content_dict[transformed_header] = content

    return content_dict


def json_to_markdown(content_dict: dict) -> str:
    """
    Reconstructs a markdown string from a dictionary where keys are headers
    and values are paragraph text.

    Args:
        content_dict: A dictionary with transformed headers as keys
                             and content as values.

    Returns:
        The reconstructed markdown string.
    """
    markdown_lines = []

    for header, content in content_dict.items():
        # Transform the key back into a header: replace underscores with spaces, capitalize
        original_header = header.replace("_", " ").title()
        # Add the header and its content in markdown format
        markdown_lines.append(f"## {original_header}")
        markdown_lines.append(content)
        markdown_lines.append("")  # Add a blank line for spacing

    # Join the lines to form the complete markdown text
    return "\n".join(markdown_lines)

Optionally, we can populate the storage with some Thought Templates.

In [46]:
# | export

user_id = "user_123"
memory = "thought_templates"

populate = False
# Optional, we populate the memory with some templates
if populate:
    template_path = Path("data") / memory
    for md_fl in template_path.iterdir():
        template_name, _ = md_fl.name.split(".")
        with open(md_fl) as fl:
            template_store.put(
                (user_id, memory), template_name, markdown_to_json(fl.read())
            )

## States

In [47]:
# | export

class BoTState(MessagesState):
    template_text: Optional[str]
    distilled_problem: str

## Nodes

In [ ]:
# | export

def problem_distiller(state: BoTState) -> dict:
    distilled_problem = LLM.invoke(
        [problem_distiller_prompt, state['messages'][-1]])
    return {
        'distilled_problem': distilled_problem.content,
        'messages': distilled_problem
    }


def template_retrieval(state: BoTState, config: RunnableConfig, store: BaseStore) -> dict:
    items = store.search(
        (user_id, memory), query=state["distilled_problem"], limit=1
    )
    template = items.pop() if items else None
    if template and template.score > config['configurable']['retrieval_threshold']:
        template_text = json_to_markdown(
            {
                'solution_description': template.value['solution_description'],
                'thought_template': template.value['thought_template']
            }
        )
        return {'template_text': template_text}
    else:
        return {'template_text': None}


def instantiate_reasoning(state: BoTState, config: RunnableConfig, store: BaseStore) -> dict:
    if state['template_text']:
        return {
            'messages': LLM.invoke([state['distilled_problem'], instantiate_reasoning_prompt + f"\n<thought-template>\n{state['template_text']}\n<thought-template>"])
        }
    else:
        return {
            'messages': LLM.invoke([state['distilled_problem'], instantiate_reasoning_prompt])
        }


def template_distillation(state: BoTState, config: RunnableConfig, store: BaseStore) -> dict:
    items = store.search(
        (user_id, memory), query=state["messages"][-1].content, limit=100
    )
    in_task_l = [template for template in items if template.score >
                 config['configurable']['in_task_threshold']]
    in_task = in_task_l.pop() if in_task_l else None
    cross_task_l = [template for template in items if template.score <= config['configurable']
                    ['in_task_threshold'] and template.score > config['configurable']['cross_task_threshold']]
    cross_task = cross_task_l.pop() if cross_task_l else None
    return {'messages': LLM.invoke(template_distiller_prompt.format(
        distilled_problem=state['distilled_problem'],
        solution_steps=state['messages'][-1].content,
        in_task_examples=json_to_markdown(in_task.value if in_task else {}),
        cross_task_examples=json_to_markdown(
            cross_task.value if cross_task else {})
    ))}


def dynamic_meta_buffer_update(state: BoTState, config: RunnableConfig, store: BaseStore) -> dict:
    # Invoke the extractor
    result = structure_template_text.invoke(
        {"messages": [structure_prompt.format(state['messages'][-1].content)]})

    r, rmeta = result["responses"].pop(), result["response_metadata"].pop()

    items = store.search(
        (user_id, memory), query=r.task_description, limit=1
    )

    template = items.pop() if items else None
    if template and template.score < config['configurable']['update_threshold']:
        store.put((user_id, memory),
                  rmeta.get("json_doc_id", str(uuid.uuid4())),
                  r.model_dump(mode="json"),
                  )
        return {'messages': 'Meta Buffer updated'}
    else:
        return {'messages': 'No update'}